<a href="https://colab.research.google.com/github/oddgai/tensorflow_tutorial/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow 2.0 クイックスタート
https://www.tensorflow.org/tutorials/quickstart/beginner?hl=ja

In [22]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

In [3]:
tf.__version__

'2.5.0'

In [11]:
# MNISTデータセットのロード
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 整数から小数に変換
x_train, x_test = x_train / 255.0, x_test / 255.0

In [48]:
# モデル構築
model = Sequential([
  Flatten(input_shape=(28, 28)),  # 入力層, 28x28の画像をflattenする
  Dense(128, activation="relu"),  # 128次元ニューロン層
  Dropout(0.2),  # 2割のニューロンをランダムに削除
  Dense(10, activation="softmax")  # 0-9を判別, 値を確率化
], name="mnist_model")

In [49]:
model.summary()

Model: "mnist_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               100480    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [50]:
# トレーニング設定
model.compile(optimizer="adam",  # 最適化アルゴリズム
              loss="sparse_categorical_crossentropy",  # 損失関数
              metrics=["accuracy"]  # 表示する項目
)

In [51]:
history = model.fit(
    x_train, 
    y_train, 
    batch_size=128, # 1回の学習で読み込む画像数
    epochs=10,  # 全件学習を何回回すか
    validation_split=0.2  # データの後ろ2割を検証データにする
)

Epoch 1/10
375/375 [==============================] - 2s 4ms/step - loss: 0.4444 - accuracy: 0.8731 - val_loss: 0.2079 - val_accuracy: 0.9442
Epoch 2/10
375/375 [==============================] - 1s 3ms/step - loss: 0.2109 - accuracy: 0.9394 - val_loss: 0.1591 - val_accuracy: 0.9552
Epoch 3/10
375/375 [==============================] - 1s 3ms/step - loss: 0.1611 - accuracy: 0.9540 - val_loss: 0.1295 - val_accuracy: 0.9626
Epoch 4/10
375/375 [==============================] - 1s 3ms/step - loss: 0.1341 - accuracy: 0.9609 - val_loss: 0.1120 - val_accuracy: 0.9669
Epoch 5/10
375/375 [==============================] - 1s 3ms/step - loss: 0.1136 - accuracy: 0.9669 - val_loss: 0.1032 - val_accuracy: 0.9698
Epoch 6/10
375/375 [==============================] - 1s 3ms/step - loss: 0.0985 - accuracy: 0.9712 - val_loss: 0.0950 - val_accuracy: 0.9727
Epoch 7/10
375/375 [==============================] - 1s 3ms/step - loss: 0.0863 - accuracy: 0.9737 - val_loss: 0.0910 - val_accuracy: 0.9727
Epoch 

In [52]:
model.evaluate(x_test,  y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.0767 - accuracy: 0.9772


[0.07671228796243668, 0.9771999716758728]

In [63]:
# 推論
test_data = x_test[0]
test_label = y_test[0]

# バッチ処理を前提としているため、1枚でも(28,28) -> (1,28,28)にしないといけない
predict = model.predict(test_data[None, ...])
print(f"正解: {test_label}")
print(f"予測: {predict[0].argmax()}, 確度 {max(predict[0])}")

正解: 7
予測: 7, 確度 0.9997331500053406


In [71]:
def sample_predict(test_img, test_label):
    predict = model.predict(test_img[None, ...])
    print(f"{test_label} : {predict[0].argmax()}, 確度 {max(predict[0])}")

In [74]:
import random

for _ in range(10):
    r = random.randint(0, len(x_test))
    test_img = x_test[r]
    test_label = y_test[r]
    sample_predict(test_img, test_label)

1 : 1, 確度 0.9998078942298889
7 : 7, 確度 0.9999316930770874
2 : 2, 確度 0.999752938747406
0 : 0, 確度 0.9999547004699707
7 : 7, 確度 0.9995267391204834
6 : 6, 確度 0.9999752044677734
6 : 6, 確度 0.9999862909317017
0 : 0, 確度 0.991617739200592
1 : 1, 確度 0.999893307685852
6 : 6, 確度 0.9999425411224365


学び
* モデル構築は`Dense`, `Dropout`とかを重ねればいける
* トレーニング設定`model.compile()`のときにロス関数とかを問題に合わせて設定しないとよくわからない結果になる
* 学習後の推論は1次元増やす必要がある